In [100]:
from google.colab import files
uploaded = files.upload()
import io


Saving boston.csv to boston.csv


# Pregunta 1 
## Prepare el ambiente de trabajo

In [1]:
import pandas as pn
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

# df = pn.read_csv('boston.csv', index_col=0)
df = pd.read_csv(io.BytesIO(uploaded['boston.csv']), encoding='latin-1', index_col=0)
df.head()

NameError: name 'pd' is not defined

# Pregunta 2
## División de la muestra

In [0]:
y_vec= df.loc[:, 'medv']
x_mat = df.drop(columns=['medv'])
x_train, x_test, y_train, y_test = train_test_split(x_mat, y_vec, test_size=0.33, random_state=11238)

# Preunta 3
##  Generación de modelos

In [0]:
modelo1 = linear_model.LinearRegression(fit_intercept=True, normalize=True)
modelo2 = linear_model.LinearRegression(fit_intercept=False, normalize=False)

no_normalizado = modelo1.fit(x_train, y_train)
normalizado = modelo2.fit(x_train, y_train)

In [0]:
medv_yhat_no_normalizado = modelo1.predict(X_test)

In [0]:
medv_yhat_normalizado = modelo2.predict(X_test)

# Pregunta 4
## Obtención de métricas

In [131]:
def report_scores(por_validar, predichos):
    m1_mse = mean_squared_error(por_validar, predichos).round(1)
    m1_r2 = r2_score(por_validar, predichos).round(2)
    print("Mean Squared Error: ", m1_mse)
    print("R-cuadrado: ", m1_r2)

print ('---Modelo no normalizado---')
report_scores(y_test, medv_yhat_no_normalizado)

print ('---Modelo normalizado---')
report_scores(y_test, medv_yhat_normalizado)

---Modelo no normalizado---
Mean Squared Error:  30.7
R-cuadrado:  0.6
---Modelo normalizado---
Mean Squared Error:  34.3
R-cuadrado:  0.55


El modelo no normalizado presenta un menor promedio de error cuadrado, posee mayor capacidad de explicar el comportamiento de nuestra variable dependiente y un $R^2$ mayor, explicando mejor el conjunto de atributos

# Pregunta 5
## Refactorización del modelo

In [132]:
def fetch_features (df,vector = 'medv'):
    columns = df.columns
    nombre = []
    r_pearson = []
    r_corregido = []
    for col in columns:
            nombre.append(col)
            r_pearson.append(df[col].corr(df[vector]))
            r_corregido.append(abs(df[col].corr(df[vector])))
    resultados = pd.DataFrame ({"nombre":nombre,
                      "r_pearson": r_pearson,
                       "r_corregido": r_corregido})
    resultados = resultados.set_index("nombre")
    return resultados
 
fetch_features (df, "medv")

,r_pearson,r_corregido
nombre,,
crim,-0.388305,0.388305
zn,0.360445,0.360445
indus,-0.483725,0.483725
chas,0.175260,0.175260
nox,-0.427321,0.427321
rm,0.695360,0.695360
age,-0.376955,0.376955
dis,0.249929,0.249929
rad,-0.381626,0.381626


    Los principales resultados son las correlaciones de medv con:
    - Istat con r_corregido de 0.73
    - rm con r_corregido de 0.69
    - ptratio con r_corregido de 0.50
    - indus con r_corregido de 0.48
    - tax con r_corregido de 0.46
    - nox con r_corregido de 0.42

# Pregunta 6: 
## Refactorización del modelo predictivo

In [139]:
df_nuevo = df.loc[:, ["medv","indus","nox", "rm", "tax", "ptratio", "lstat"]]
y_vec2 = df_nuevo["medv"]
x_mat2 = df_nuevo.loc[:, ["indus","nox", "rm", "tax", "ptratio", "lstat"]] 
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_mat2, y_vec2, test_size=.33, random_state=11238)
modelo3 = LinearRegression(fit_intercept=True, normalize=True)
nuevo_modelo_normalizado = modelo3.fit(x_train2, y_train2)
purge_yhat = nuevo_modelo_normalizado.predict(x_test2)
print ('---Modelo refactorizado, normalizado---')
report_scores(y_test, purge_yhat)


modelo4 = LinearRegression(fit_intercept=False, normalize=False)
no_normalizado = modelo1.fit(x_train2, y_train2)
purge_yhat = no_normalizado.predict(x_test2)
print ('---Modelo refactorizado, no normalizado---')
report_scores(y_test, purge_yhat)

---Modelo refactorizado, normalizado---
Mean Squared Error:  37.5
R-cuadrado:  0.51
---Modelo refactorizado, no normalizado---
Mean Squared Error:  37.5
R-cuadrado:  0.51


El mejor modelo es el refactorizado, no normalizado. 

# Desafío 7 
## Predicción de casos

In [143]:
worst_neighbor = np.array([37.9, 12.6, 3.5, 27.7, 187, 0.87]).reshape(1, -1)
best_neighbor = np.array([1.73, 22, 8.7, 0.46, 711, 0.38]).reshape(1, -1)

worst_neighbor = np.array([27.7, 0.87, 3.5, 187, 12.6 ,37.9]).reshape(1, -1)
best_neighbor = np.array([0.46, 0.38, 8.7, 711, 22, 1.73]).reshape(1, -1)

purge_yhat = no_normalizado.predict(best_neighbor)
purge_yhat

array([35.31255605])

In [144]:
purge_yhat = no_normalizado.predict(worst_neighbor)
purge_yhat

array([3.01572229])